In [42]:
import numpy as np
from itertools import combinations
import pandas as pd

# Функция для кодирования информационного вектора
def encode(a):
    hemm = list('0000000')
    stepeni_dvoiki = [0, 1, 3]
    x = 3
    for i in range(len(hemm)):
        if i in stepeni_dvoiki:
            hemm[i] = a[0]
            if i == 0:
                hemm[i] = str(int(hemm[i]) ^ int(a[2]) ^ int(a[3]))
            elif i == 3:
                hemm[i] = str(int(hemm[i]) ^ int(a[2]) ^ int(a[1]))
            else:
                hemm[i] = str(int(hemm[i]) ^ int(a[1]) ^ int(a[3]))
        else:
            hemm[i] = a[x]
            x -= 1

    return list(map(int, hemm[::-1]))

# Функция для вычисления синдрома ошибки
def syndrom_err(vector):
    h1 = vector[0] ^ vector[2] ^ vector[4] ^ vector[6]
    h2 = vector[1] ^ vector[2] ^ vector[5] ^ vector[6]
    h3 = vector[3] ^ vector[4] ^ vector[5] ^ vector[6]
    return [h3, h2, h1]

# Генерация всех векторов ошибки
def generate_error_vectors(n):
    error_vectors = []
    for weight in range(1, n + 1):  # От 1 до n ошибок
        for positions in combinations(range(n), weight):
            error = np.zeros(n, dtype=int)
            error[list(positions)] = 1
            error_vectors.append(error)
    return error_vectors

# Постоянные параметры кода Хэмминга [7, 4]
n = 7  # Длина кодового слова
k = 4  # Длина информационного слова

# Функция для расчета корректирующей и обнаруживающей способностей
def calculate_capabilities_with_info_vector(info_vector, G, H, n):
    codeword = encode(info_vector)
    print("Информационный вектор:", info_vector)
    print("Закодированный вектор:", codeword)

    error_vectors = generate_error_vectors(n)
    
    dict_naiden = {i: 0 for i in range(1, n + 1)}  # Кратность ошибок от 1 до n
    dict_all_err = {i: 0 for i in range(1, n + 1)}
    dict_correct = {i: 0 for i in range(1, n + 1)}

    for error in error_vectors:
        error_weight = sum(error)
        dict_all_err[error_weight] += 1

        # Применяем ошибку
        received = (codeword + error) % 2

        # Рассчитываем синдром и пытаемся исправить
        syndrom = syndrom_err(received)
        if sum(syndrom) != 0:  # Ошибка обнаружена
            dict_naiden[error_weight] += 1
            hemm_err = received.copy()
            syndrom_decimal = int("".join(map(str, syndrom)), 2)
            if syndrom_decimal > 0:
                hemm_err[syndrom_decimal - 1] ^= 1
            if np.array_equal(hemm_err, codeword):  # Ошибка исправлена
                dict_correct[error_weight] += 1

    # Формируем DataFrame с результатами
    data = {
        'Кратность ошибки': list(dict_all_err.keys()),
        'Все ошибки данной кратности': list(dict_all_err.values()),
        'Исправленные': [dict_correct[i] for i in range(1, n + 1)],
        'Найденные': [dict_naiden[i] for i in range(1, n + 1)]
    }
    df = pd.DataFrame(data)
    df['Корректирующая способность кода'] = df['Исправленные'] / df['Все ошибки данной кратности']
    df['Обнаруживающая способность кода'] = df['Найденные'] / df['Все ошибки данной кратности']
    
    return df

# Исходный информационный вектор для варианта 12
info_vector = np.array([1, 0, 1, 1], dtype=int)
# Расчет и вывод результатов
df_results_with_info = calculate_capabilities_with_info_vector(info_vector, G=None, H=None, n=n)

df_results_with_info


Информационный вектор: [1 0 1 1]
Закодированный вектор: [1, 0, 1, 0, 1, 0, 1]


,Кратность ошибки,Все ошибки данной кратности,Исправленные,Найденные,Корректирующая способность кода,Обнаруживающая способность кода
0,1,7,7,7,1.0,1.0
1,2,21,0,21,0.0,1.0
2,3,35,0,28,0.0,0.8
3,4,35,0,28,0.0,0.8
4,5,21,0,21,0.0,1.0
5,6,7,0,7,0.0,1.0
6,7,1,0,0,0.0,0.0
